# Data Processing

This workflow loads the underlying raw data and processes it to address any underlying issues and optimize our data for analysis.

The workflow will create cleaned datasets for other workflows to reuse.

In [18]:
import pandas as pd
from functools import reduce
from collections import Counter

## Data Import
Our analysis includes the following data sets:

In [19]:
life_expectancy = pd.read_csv('../data/life_expectancy.csv', index_col = 'country')
population = pd.read_csv('../data/population.csv', index_col = 'country')
gni = pd.read_csv('../data/gni_per_capita.csv', index_col = 'country')

gni_title = "Per Capita GNI (Gross National Income)"
life_expectancy_title = "Life Expectancy"
population_title = "Population"

datasets = {
    gni_title: gni,
    life_expectancy_title: life_expectancy,
    population_title: population
}

In [20]:
def print_data_sets():
    print(f'Data Sets ({len(datasets)}):\n')
    def describe_data(index: int, title: str, data: pd.DataFrame)-> str:
        return f"""
        {index + 1}. {title}
        {len(data)} countries from {data.columns[0]} to {data.columns[-1]}

        {data.head(5)}
        """
    descriptions = '\n---\n'.join([describe_data(index, title, data) for index, (title, data) in enumerate(datasets.items())])
    print(descriptions)
print_data_sets()

Data Sets (3):


        1. Per Capita GNI (Gross National Income)
        191 countries from 1800 to 2050

                               1800   1801   1802   1803   1804   1805   1806   1807  \
country                                                                        
Afghanistan           207.0  207.0  207.0  207.0  207.0  207.0  207.0  207.0   
Angola                517.0  519.0  522.0  524.0  525.0  528.0  531.0  533.0   
Albania               207.0  207.0  207.0  207.0  207.0  207.0  207.0  207.0   
United Arab Emirates  738.0  740.0  743.0  746.0  749.0  751.0  754.0  757.0   
Argentina             794.0  797.0  799.0  802.0  805.0  808.0  810.0  813.0   

                       1808   1809  ...   2041   2042   2043   2044   2045  \
country                             ...                                      
Afghanistan           207.0  207.0  ...    751    767    783    800    817   
Angola                536.0  537.0  ...   2770   2830   2890   2950   3010   
Albania    

## Data Cleaning
Analysis of the underlying data shows the following issues:

#### 1. Missing Years
The following countries have years which are missing data:

In [21]:
def print_missing_years():
    for name, df in datasets.items():
        year_count = len(df.columns)
        # per-country user-facing description of missing years
        # e.g.: '231 of 301'
        missing_years = df.isnull().sum(axis=1)[lambda x: x > 0].map(lambda x: f'{x} of {year_count}')
        missing_years.name = "Number of Missing Years"
        # to do: show missing years as ranges instead. E.g. 1800-1850, 2050-2100
        if (count := len(missing_years)) > 0:
            print(f"""{name} data: {count} {'country' if count == 1 else 'countries' } with missing years
        
            {pd.DataFrame(missing_years)}
            """)
print_missing_years()

Per Capita GNI (Gross National Income) data: 1 country with missing years
        
                          Number of Missing Years
country                              
Liechtenstein              229 of 251
            
Life Expectancy data: 9 countries with missing years
        
                                Number of Missing Years
country                                    
Andorra                          231 of 301
Dominica                         231 of 301
St. Kitts and Nevis              231 of 301
Monaco                           231 of 301
Marshall Islands                 231 of 301
Nauru                            231 of 301
Palau                            231 of 301
San Marino                       231 of 301
Tuvalu                           231 of 301
            
Population data: 1 country with missing years
        
                     Number of Missing Years
country                         
Holy See              100 of 301
            


#### 2. Diverging Countries and Years
Data sets differ slightly in which countries and year ranges they represent:

In [22]:
def find_inconsistencies(list_of_lists) -> set:
    # given multiple collections, find the elements that are not shared by all collections
    if len(list_of_lists) == 0:
        return []
    sets = [set(collection) for collection in list_of_lists]
    all_elements = reduce(set.union, sets, set({}))
    common_elements = reduce(set.intersection, sets, sets[0])
    return sorted(list(all_elements - common_elements))

def inconsistent_countries():
    return find_inconsistencies([data.index for data in datasets.values()])

def inconsistent_years():
    return find_inconsistencies([data.columns for data in datasets.values()])

def print_inconsistent_countries():
    print(f'Countries not present in all data sets:')
    print(inconsistent_countries())
    
def print_inconsistent_years():
    print(f'\nYears not present in all data sets:')
    years = inconsistent_years()
    if len(years) > 0:
        # missing years happen to represent a range
        print(f'{years[0]}...{years[-1]}')

print_inconsistent_countries()
print_inconsistent_years()

Countries not present in all data sets:
['Andorra', 'Holy See', 'Liechtenstein', 'Monaco', 'North Korea', 'San Marino', 'Taiwan']

Years not present in all data sets:
2051...2100


### 3. Data Types
For our analysis, we expect all data to be numeric. However, we unexpectedly found large volumes of non-numeric data:

In [23]:
def describe_counts(counts: Counter) -> str:
    tuples = counts.most_common()
    count_descriptions = map(lambda tuple: f'{str(tuple[0])}({tuple[1]})', tuples)
    return ', '.join(count_descriptions)
    
# First, determine the mix of data types present in each data set
def print_data_types():
    print("Data Types (expected: all numeric)\n")
    for name, data in datasets.items():
        columntypes = Counter(data.dtypes.values)
        print(f'{name} Columns: {describe_counts(columntypes)}')
        print(f'{name} Column Index: {data.columns.dtype}')
        datatypes = data.map(type).apply(lambda column: Counter(column))
        overall_counts = Counter()
        for counter in datatypes:
            overall_counts += counter  
        print(f'{name} Values: {describe_counts(overall_counts)}')

# We expected only numeric data but found mixed data types.
# Let's surface all non-numeric values to establish a root cause.
def print_malformed_data():
    print('\n\nMalformed data examples:\n')    
    for name, data in datasets.items():
        print(name, ':')
        # list of non-numeric values per country
        # this is unexpectedly nontrivial because countries contain numeric values encoded as strings
        bad_values_by_country = data.apply(lambda row: [x for x in row if type(x) == str and not x.replace('.', '').isdigit()], axis = 1)
        # convert list to string and filter out empty lists
        bad_values_by_country = bad_values_by_country.apply(lambda values: None if len(values) == 0 else ', '.join(values)).dropna()
        print("No malformed data" if bad_values_by_country.count() == 0 else bad_values_by_country)
        if name != 'Population':
            print('\n---\n')

print_data_types()
print_malformed_data()


Data Types (expected: all numeric)

Per Capita GNI (Gross National Income) Columns: object(154), float64(97)
Per Capita GNI (Gross National Income) Column Index: object
Per Capita GNI (Gross National Income) Values: <class 'str'>(29282), <class 'float'>(18659)
Life Expectancy Columns: float64(301)
Life Expectancy Column Index: object
Life Expectancy Values: <class 'float'>(58695)
Population Columns: object(301)
Population Column Index: object
Population Values: <class 'str'>(59197), <class 'float'>(100)


Malformed data examples:

Per Capita GNI (Gross National Income) :
country
Albania                           10k, 10.2k, 10.5k, 10.7k, 10.9k, 11.1k, 11.4k,...
United Arab Emirates              10k, 10.5k, 10.9k, 11.3k, 11.7k, 11.4k, 11.1k,...
Argentina                         10.3k, 12k, 12.6k, 11.7k, 12.1k, 12.8k, 12.7k,...
Antigua and Barbuda               10k, 10.9k, 11.7k, 12.3k, 12.8k, 12.7k, 13.4k,...
Australia                         10.4k, 11.5k, 11.8k, 11.3k, 10.6k, 10.1k, 10

The underlying non-numeric data comes from a widely used convention of using abbreviated numeric strings such as '123K' or '4.5M'. 

## Data Modifications
Based on investigating our data, we will apply the following modifications to prepare our data for analysis:

- **Restrict dates to 1923-2023**. This solves the issue of inconsistent date ranges between data sets and avoids the issue of combining recorded data with projected data.
- **Remove inconsistent countries** that are not present in all data sets. Keep only countries that have complete data.
- **Convert abbreviated numbers** such as '1K' to numeric values such as 1_000. This substantially helps us in analyzing our data, since our tooling does not understand numbers such as '1K'.
- **Convert column names to numeric**. Per-year column names are interpreted as strings, but should be numeric instead.
- **Convert all column data types to numeric**. Note that in Pandas, converting column data types is a separate step from converting the underlying values.
- **No changes for missing years**: Some countries with missing years may be especially interesting due to their unique histories, and our analysis is robust against missing values. So no changes are needed for addressing missing values.

### Handling Numeric Strings

Let's create a utility to handle widespread use of shorthand numeric strings by converting them to numbers.

In [24]:
def parse_number(value):
    if type(value) in (int, float):
        return value

    # Given an abbreviated numeric string such as '23K' or '1.2M', returns a resolved number such as 23_000 or 1_200_000
    # Returns None if the input string is not an abbreviated numeric string
    try:
        abbreviations = {
            "K":1_000,
            "M":1_000_000,
            "B":1_000_000_000,
            "T":1_000_000_000_000
        }
        last_char = value[-1].upper()
        # if numeric string ends in thousands/millions/etc, apply the multiplier
        if (multiplier := abbreviations.get(last_char)) != None:
            remainder = value[:-1]
            return int(multiplier * parse_number(remainder))
        # otherwise, perform a regular conversion
        else:
            is_decimal = '.' in value
            return float(value) if is_decimal else int(value)
    except:
        return None        

Let's test the utility as thoroughly as possible to ensure it is bulletproof:

In [25]:
strings = ['1', '-1', '1.', '.1', '3.1415', '10K', '1.23k', '2.0001M', '9B', '237T', '1_234', '1.23456k', '1_23', '1_1_1', '1 K', ' 2 t', '1_000m', '1km']
numbers = [1, 0.1, -0, -234]
invalid_strings = ['', '.', '_1', 'M', '.M', '3X', 'M3', '.T', '1,234', '1e10']
for value in strings + numbers + invalid_strings:
    result = parse_number(value)
    if result == None:
        print(f'parse_number({repr(value)}) = None')
    else:
        print(f'parse_number({repr(value)}) = {result:,}')

parse_number('1') = 1
parse_number('-1') = -1
parse_number('1.') = 1.0
parse_number('.1') = 0.1
parse_number('3.1415') = 3.1415
parse_number('10K') = 10,000
parse_number('1.23k') = 1,230
parse_number('2.0001M') = 2,000,100
parse_number('9B') = 9,000,000,000
parse_number('237T') = 237,000,000,000,000
parse_number('1_234') = 1,234
parse_number('1.23456k') = 1,234
parse_number('1_23') = 123
parse_number('1_1_1') = 111
parse_number('1 K') = 1,000
parse_number(' 2 t') = 2,000,000,000,000
parse_number('1_000m') = 1,000,000,000
parse_number('1km') = 1,000,000,000
parse_number(1) = 1
parse_number(0.1) = 0.1
parse_number(0) = 0
parse_number(-234) = -234
parse_number('') = None
parse_number('.') = None
parse_number('_1') = None
parse_number('M') = None
parse_number('.M') = None
parse_number('3X') = None
parse_number('M3') = None
parse_number('.T') = None
parse_number('1,234') = None
parse_number('1e10') = None


Finally, let's put it all together by transforming our raw data:

In [26]:
start_year = 1923
end_year = 2023

print ('Transforming data...')
print('Converting date columns to numeric...')
print(f'Restricting date ranges to {start_year}...{end_year}...')
print('Converting abbreviated numbers such as \'1K\'...')
for name, data in datasets.items():
     # Convert date columns to int
    data.columns = data.columns.astype(int)
    # Restrict date range
    removed_years = data.columns.difference(list(range(start_year,end_year + 1)))
    data.drop(columns = removed_years, inplace = True)
    # Remove inconsistent countries
    removed_countries = data.index.intersection(inconsistent_countries())
    if not removed_countries.empty:
        print(f'removing countries {removed_countries.values} from {name} data')
        data.drop(index = removed_countries, inplace = True)
    # Convert abbreviated numbers
    data[:] = data.map(parse_number)
    # data[:] = data.astype(float, copy = False)  # does not work for unknown reasons. This invalidates the whole idea of making this method an in-place edit.
    datasets[name] = data.astype(float, copy = False) # undesirable because this does create a copy of the original instead of editing in-place
# restore link between global data sets and data sets within the dictionary because the previous phase created copies    
gni = datasets[gni_title]
population = datasets[population_title]
life_expectancy = datasets[life_expectancy_title]

# Show cleaned data
print('Finished transforming data\n')
print_data_types()
print('\n')
print_inconsistent_countries()
print('\n')
print_malformed_data()
print('\n')
print_data_sets()


Transforming data...
Converting date columns to numeric...
Restricting date ranges to 1923...2023...
Converting abbreviated numbers such as '1K'...
removing countries ['Liechtenstein'] from Per Capita GNI (Gross National Income) data
removing countries ['Andorra' 'Monaco' 'North Korea' 'San Marino' 'Taiwan'] from Life Expectancy data
removing countries ['Andorra' 'Holy See' 'Liechtenstein' 'Monaco' 'North Korea' 'San Marino'
 'Taiwan'] from Population data
Finished transforming data

Data Types (expected: all numeric)

Per Capita GNI (Gross National Income) Columns: float64(101)
Per Capita GNI (Gross National Income) Column Index: int64
Per Capita GNI (Gross National Income) Values: <class 'float'>(19190)
Life Expectancy Columns: float64(101)
Life Expectancy Column Index: int64
Life Expectancy Values: <class 'float'>(19190)
Population Columns: float64(101)
Population Column Index: int64
Population Values: <class 'float'>(19190)


Countries not present in all data sets:
[]




Malformed

## Save Results
Save the cleaned datasets so that other workflows can leverage the data cleaning workflow.

In [27]:
gni.to_csv('../data/gni_per_capita_cleaned.csv')
life_expectancy.to_csv('../data/life_expectancy_cleaned.csv')
population.to_csv('../data/population_cleaned.csv')


Let's validate the work by ensuring that we can easily load cleaned data:

In [28]:
datasets = {
    gni_title: pd.read_csv('../data/gni_per_capita_cleaned.csv', index_col='country'),
    life_expectancy_title: pd.read_csv('../data/life_expectancy_cleaned.csv', index_col='country'),
    population_title: pd.read_csv('../data/population_cleaned.csv', index_col='country')
}
print_data_types()

Data Types (expected: all numeric)

Per Capita GNI (Gross National Income) Columns: float64(101)
Per Capita GNI (Gross National Income) Column Index: object
Per Capita GNI (Gross National Income) Values: <class 'float'>(19190)
Life Expectancy Columns: float64(101)
Life Expectancy Column Index: object
Life Expectancy Values: <class 'float'>(19190)
Population Columns: float64(101)
Population Column Index: object
Population Values: <class 'float'>(19190)


In [34]:
def convert_wide_data_to_long(*datasets, dataset_names=None):
    """
    Combines multiple datasets in crosstab format (years as columns, countries as rows) into a single dataset
    with features as columns.
    
    Parameters
    ----------
    *datasets : pandas.DataFrame
        One or more dataframes in crosstab format with countries as index and years as columns.
    dataset_names : list, optional
        Names to use for each dataset's feature column. If None, uses the variable names from the input.
        
    Returns
    -------
    pandas.DataFrame
        A combined dataframe with columns: 'country', 'year', and one column for each input dataset.
        
    Examples
    --------
    >>> gni_per_capita = pd.DataFrame({2000: [1000, 2000], 2001: [1100, 2100]}, index=['CountryA', 'CountryB'])
    >>> life_expectancy = pd.DataFrame({2000: [70, 75], 2001: [71, 76]}, index=['CountryA', 'CountryB'])
    >>> data = create_column_data(gni_per_capita, life_expectancy)
    >>> data.columns
    Index(['country', 'year', 'gni_per_capita', 'life_expectancy'], dtype='object')
    """
    # If dataset names not provided, use their variable names as column names
    if dataset_names is None:
        # Try to get variable names from calling context
        import inspect
        frame = inspect.currentframe().f_back
        args_info = inspect.getframeinfo(frame).code_context[0]
        args_str = args_info[args_info.find('(') + 1:args_info.find(')')]
        arg_names = [arg.strip() for arg in args_str.split(',')]
        # Filter out any keyword arguments
        arg_names = [name for name in arg_names if '=' not in name]
        # Use only as many names as we have datasets
        dataset_names = arg_names[:len(datasets)]
        print(dataset_names)
    
    # Ensure we have names for all datasets
    if len(dataset_names) != len(datasets):
        dataset_names = [f'feature_{i}' for i in range(len(datasets))]
    
    # Create a list to hold all the transformed dataframes
    transformed_dfs = []
    
    # Process each dataset
    for df, name in zip(datasets, dataset_names):
        # Reset index to make country a column
        df_reset = df.reset_index()
        
        # Melt the dataframe to convert years from columns to rows
        df_melted = pd.melt(
            df_reset, 
            id_vars='country',
            var_name='year',
            value_name=name
        )
        
        # Convert year to numeric
        df_melted['year'] = pd.to_numeric(df_melted['year'])
        
        # Add to our list
        transformed_dfs.append(df_melted)
    
    # Merge all dataframes on country and year
    result = transformed_dfs[0]
    for df in transformed_dfs[1:]:
        result = pd.merge(result, df, on=['country', 'year'])
    
    return result


In [35]:
combined_data = convert_wide_data_to_long(gni, life_expectancy, population)
combined_data.head()

['gni', 'life_expectancy', 'population']


,country,year,gni,life_expectancy,population
0,Afghanistan,1923,373.0,37.7,9370000.0
1,Angola,1923,730.0,37.0,2940000.0
2,Albania,1923,447.0,34.5,957000.0
3,United Arab Emirates,1923,9550.0,39.0,58300.0
4,Argentina,1923,3320.0,52.7,9610000.0


In [36]:
combined_data.to_csv('../data/world_development_data.csv')